# EE 120 Lab 5: Deconvolution

**Signals and Systems** at UC Berkeley

Acknowledgements:

- **Spring 2019** (v1.0): Dominic Carrano, Ilya Chugunov, Babak Ayazifar  
- **Fall 2019** (v2.0): Dominic Carrano, Ilya Chugunov  
- **Spring 2020** (v2.1): Dominic Carrano, Ilya Chugunov
- **Spring 2021** (v2.2): Jingjia Chen


In [ ]:
import IPython.display as ipd
import numpy as np
import matplotlib.pyplot as plt
import scipy.signal
from scipy import signal, linalg
from scipy.io import wavfile
%matplotlib inline

# Background

Note: For simplicity, we'll consider all signals and systems in this lab as discrete-time entities, since that's what computers use. In truth, there are other steps involved, such as sampling and quantization. In class, we denote discrete time signal with square bracket $x[n]$, in this lab, since everything will be in discrete time regime, $x(n)$ also represents discrete time signal, and $X(\omega)$ is to represent the corresponding DTFT.

### Motivation: Signal Restoration

*Degradation* (also referred to as *distortion*) is a common problem in signal and image processing: you want access to a "ground truth" signal $x(n)$, but due to some real-world physical constraints, you can only observe $y(n)$, a corrupted version of $x(n)$. 

For example, suppose you're playing a song $x(n)$ in a large concert hall. Due to the acoustics of the room, your audience hears $y(n)$, the song superimposed with a quieter echo of it (that is, $x(n)$ plus a delayed and attenuated copy of $x(n)$).

In this lab we address how to compensate for signal distortions. Our goal is to produce $\hat{x}(n)$, an estimate of the original signal $x(n)$, using the measured signal $y(n)$. The process of determining $\hat{x}(n)$ using $y(n)$ is called *signal restoration*.

### Problem Formulation

From a block-diagram perspective, we can think of this *degradation* as a *system* $H$, which outputs $y(n)$, our measured signal, from input $x(n)$, the true signal. 

<img src="figs/degrade.png" width="700px"></img>

Our job is to design a *restoration system*, $R$, that attempts to undo the effects of $H$ to give us $\hat{x}(n)$, a nice approximation of $x(n)$. Using the concert hall example from above, $H$ could be the acoustics of the room, and $R$ could be a specialized filter chip in a de-echoing concert microphone you bought from your local microphone dealer.

### Designing $R$

In general, $H$ could be *any* system (e.g., nonlinear, time-varying, etc.), and undoing it may be intractable. To simplify things, we'll assume: 
- $H$, the degradation system, 
is DT-LTI (with frequency response $H(\omega)$ and impulse response $h(n)$).
- $R$, the recovery system we choose, is DT-LTI (with frequency response $R(\omega)$ and impulse response $r(n)$).

In many scenarios of practical interest (e.g. acoustic echoes, system lag, image blur), we often can justify an approximation a real-life system as LTI. 

Since $H$ is LTI, we can describe it's behaviour as a convolution, $y(n) = (x * h)(n)$. We want to design $R$ to *undo* this convolution, performing what's called ***deconvolution***, in the sense that we want to **recover** $x(n)$. For this problem it's easier to work in the frequency domain, where our time-signal convolution just becomes frequency-response multiplication:

$$Y(\omega) = H(\omega) X(\omega),$$

and

$$\hat{X}(\omega) = R(\omega) Y(\omega) = R(\omega) H(\omega) X(\omega),$$

Algebraically, if we pick $R(\omega) = 1 / H(\omega)$, this will five us $\hat{X}(\omega) = X(\omega)$, from which we can take the inverse DTFT to recover the original signal. Effectively, we compute $\hat{X}(\omega) = Y(\omega) / H(\omega)$.

This algorithm is known as *Fourier deconvolution* (sometimes also called "inverse filtering" or "direct deconvolution"), since we perform the deconvolution directly based on the Fourier Transform(s) of the systems involved.

### Issues and Alternatives

Fourier deconvolution computes the multiplicative inverse of $H$'s frequency response, and uses it to characterize the inverse system $R$. 

This approach has two main issues:
1. $H$ may be zero at some frequency $\omega_0$, in which case the division is not well-defined. Intuitively, all content at $\omega_0$ is killed, and we're left with no information about it, so we can't invert the behavior.
2. Even if $H$ is nonzero over all frequencies, it's often very small, and so $1 / H(\omega)$ will be huge, and amplify noise that will be present in practical setups.

Due to these issues, it's more common to use *Wiener filtering* in practice for deconvolution. Wiener filtering is a more sophisticated technique that uses statistical properties of the signals and noise involved to produce better results. However, Wiener filtering and Fourier deconvolution are similar in spirit (and we don't want to go down a rabbit hole into statistical signal processing), so we'll be using Fourier in this lab.

Later in the lab, we'll explore these issues and how they can affect our signals.

# Q1: Echo Cancellation

A classic problem in signals and systems, acoustics, and digital communications is *echo cancellation*. A sender transmits a signal to someone, and they receive it, along with a delayed, attenuated copy of it. 

There are many causes of this phenomenon which you can read about in the references, including:
- Signal back reflections due to impedance mismatches in electronic circuits (see references 1, 2).
- Audio feedback in microphones (see reference 3).
- Acoustic properties of the space the signal is being transmitted in. For example, if you send a signal indoors, it may go in multiple directions, with part of the signal going straight to a receiver, and the rest of it bouncing off of several walls before it arrives as a delayed and attenuated copy of the first.

## Modelling an echo

Many different models for echoes have been considered, and you can find a more comprehensive treatment of this subject in references 4 and 5. Here, we'll consider one of the simpler models for an echo, where our communication channel (the degradation system) is an LTI system with impulse response

$$h(n) = \delta(n) + \alpha \delta(n - k)$$

where $0 < \alpha < 1$ is the attenuation of the echo and $k > 0$ is the integer delay of the echo. Assuming $0 < \alpha < 1$ means the copy is weaker than the original, and $k>0$ that the copy shows up after the original.

We can think of this as a channel that transmits the signal perfectly and instantaneously, and also with a $k$-step delay and some attenuation along an echo path. In this problem, we'll send some audio over this channel, and try to undo the corruptions it introduces using deconvolution.

## A Brief Intro to Audio Signals

In this question, we'll use ["Take on Me" by a-ha](https://www.youtube.com/watch?v=djV11Xbc914) as our test signal. We have provided it for you as a [.wav](https://en.wikipedia.org/wiki/WAV) (pronounced "wave") file. Run the cell below and have a listen! It's normal if the cell takes a few seconds to load. You'll see a graphic display pop up with a play button once it's finished loading.

In [ ]:
ipd.Audio("wavs/TakeOnMe.wav")

Now, we'll read the file as a numpy array using scipy's WAV file API. The `read` function returns two things: the sampling rate of the audio, and the signal itself.

In [ ]:
# In signal processing code, "fs" is conventionally used for sampling frequency in Hertz (cycles/second)
fs, data = wavfile.read("wavs/TakeOnMe.wav")

Typically, digital audio is sampled at 44.1 kHz, or 44100 Hz, although some more modern formats use 48 kHz. This is motivated by the fact that the human ear can only hear up to ~20 kHz. Given this, the Nyquist criterion suggests that the sampling rate should be at least 40 kHz, so some extra wiggle room is added on. We can easily verify that we're dealing with a sampling rate of 44.1 kHz.

In [ ]:
print(fs) # sampling rate in Hz

When dealing with real world data, a good first step before processing it is checking what size it is using `np.shape`, just as we did when analysing functional MRI data in Lab 1.

In [ ]:
np.shape(data)

Perhaps surprisingly, the song, when read in as a signal, is actually a 1984500-by-2 matrix! Why?

The track's runtime is 45 seconds, and with a sampling rate of 44.1 kHz, we expect a total of 

$$45\ \text{seconds} \cdot \frac{44100\ \text{samples}}{1\ \text{second}} = 1984500\ \text{samples}$$

in our data. That explains the first dimension. Why a two column matrix, though?

The reason we have two separate columns of data, rather than a single array of 1984500 samples, is due to the use of [two-channel audio](https://en.wikipedia.org/wiki/Stereophonic_sound). When you listen to music with a pair of headphones, each ear is receiving a separate audio stream, hence the need for two samples at each point in time. The same principle applies to laptops or other sound systems with two speakers. 

**What this means for us is that each channel (i.e., column of this matrix) should be processed as a distinct, 1D signal.**

As a final step before moving on, we'll crop our song to the first 10 seconds. This will make processing go much faster, and we'll still be able to hear what's going on throughout the process.

In [ ]:
n_sec = 10
data_cropped = data[:n_sec * fs, :]

As a sanity check, run the cell below to play the first 10 seconds of the song. The display should show that the file has 10 seconds of audio, and it should sound exactly the same as before.

In [ ]:
wavfile.write("wavs/cropped_TakeOnMe.wav", fs, data_cropped)
ipd.Audio("wavs/cropped_TakeOnMe.wav")

Now let's make some echoes!

## Q1a: Transmission

Implement the function `transmit` below to simulate the echo channel. As a reminder, so that you don't have to keep scrolling back and forth, we're modelling the channel as an LTI system with impulse response

$$h(n) = \delta(n) + \alpha \delta(n-k)$$

where $\alpha$ (the attentuation factor) and $k$ (the delay of the echo in samples) are provided to you as function arguments, along with the signal to transmit, $x$. Your function should return the result of transmitting $x$ over the channel, performing the parasitic convolution we'll later be trying to undo. 

**As a reminder, each audio channel should be considered as a distinct signal requiring a separate convolution when transmitting.**

#### Quantization

All audio we're working with is [quantized](https://en.wikipedia.org/wiki/Quantization) to 16 bits. After processing our signal, we have to renormalize each entry to be a 16-bit integer value, or we'll introduce new distortions to it.

After you transmit the song clip **and reassemble it into a two-channel matrix**, say `x_echoed`, apply the following line of code: 

**`np.int16(x_echoed / np.max(np.abs(x_echoed)) * np.iinfo(np.int16).max)`**. 

This fits every value to the range [-1, +1] and then rescales it to be within the range of [-32767, 32767]. This will be the last thing you have to do before returning the result.

**Hint:** When using Numpy built-in [convolution operation](https://numpy.org/doc/stable/reference/generated/numpy.convolve.html)**`np.convolve`** to perform convolution, all convolutions should be done in "full" mode to avoid cutting out data. Since we're using "full", there's no need to pad implicit zeros onto the echo channel impulse response; it should only be length $k+1$.

In [ ]:
def transmit(x, alpha, k):
    """
    Simulate transmission of a two-channel audio signal x over an LTI echo channel which sends 
    x and a copy of x delayed by k > 0 samples and attenuated by a factor 0 < alpha < 1.
    
    Parameters:
    x        - The audio signal. Assumed to be an Nx2 matrix, where N is the number of audio samples.
    alpha    - The attenuation factor. Assumed that 0 < alpha < 1.
    k        - The delay factor, in samples. Assumed that k > 0.
    
    Returns:
    x_echoed - The echoed signal.
    """
    # TODO your code here
   

Once you've finished implementing `transmit`, try it out by running the cell below, which will generate an echo at $80\%$ the strength of the original signal and with a delay of $2 \cdot f_s = 88200$ samples (exactly two seconds). 

**This means our transmitted song will be 12 seconds long.** The original copy starts at time zero and finishes 10 seconds in. The echoed copy starts 2 seconds in, and ends after 12 seconds from the start of the original copy.

This cell will take anywhere from several seconds to a minute to run depending on your laptop. Even with 10 seconds of data, we have two $10 \cdot 44100$ entry convolutions to compute, which will take some time. If it takes longer than a few minutes, your code's probably wrong.

In [ ]:
x_echo = transmit(data_cropped, .8, 2*fs)

Run the cell below to play your echo-corrupted song. 

You should hear a second copy of the track that comes in two seconds later. This means that the first and last two seconds of the audio should only contain one track. The first two seconds will contain the start of the original, and the last two seconds will be the end of the echo. It should be easy to tell if your result is correct or not by just listening.

In [ ]:
wavfile.write("wavs/echoed_TakeOnMe.wav", fs, x_echo)
ipd.Audio("wavs/echoed_TakeOnMe.wav")

## Q1b: Deconvolving

Implement the function `deconvolve` below using the Fourier deconvolution algorithm described in the background. Feel free to use NumPy's [fft](https://docs.scipy.org/doc/numpy/reference/generated/numpy.fft.fft.html#numpy.fft.fft) and [inverse fft](https://docs.scipy.org/doc/numpy/reference/generated/numpy.fft.ifft.html#numpy.fft.ifft) functions for this part.

As with Lab 3's question on oscilloscope signal alignment, we're going to encounter the issue of numerical noise here yielding an erroneously nonzero imaginary part in our final result. **Be sure to take the real part of any signal returned to avoid a fake imaginary part showing up.**

In [ ]:
def deconvolve(y, h):
    """
    Perform a Fourier deconvolution to deconvolve h "out of" y, assuming
    that h, y and the deconvolved signal are both real-valued.
    
    Parameters:
    y - The signal to deconvolve h out of.
    h - The impulse response used in the parasitic convolution.
    """
    ## TODO ##
 

As a sanity check, let's do a toy example of echo cancellation. 

We'll set:
- $x(n) = .5 \delta(n) + \delta(n-1) + .5 \delta(n-2)$, a three-point pulse.
- $h(n) = \delta(n) + .4 \delta(n - 7)$, which will generate an echo with a seven-sample delay at $40\%$ the original's strength.

Run the cell below to see what this looks like.

In [ ]:
x = np.array([.5, 1, .5]); h = np.array([1, 0, 0, 0, 0, 0, 0, .4]); y = np.convolve(x, h, "full")

x_pad = np.concatenate((x, np.zeros(len(y) - len(x))))
h_pad = np.concatenate((h, np.zeros(len(y) - len(h))))

plt.figure(figsize=(16, 4))
plt.subplot(1, 3, 1); plt.stem(x_pad); plt.title("Original signal x")
plt.subplot(1, 3, 2); plt.stem(h_pad, linefmt="C1", markerfmt="C1o"); plt.title("Impulse response h")
plt.subplot(1, 3, 3); plt.stem(y, linefmt="C2", markerfmt="C2o"); plt.title("Echoed version y = h * x")
plt.show()

We want to remove that second pulse that shows up in $y$. 

This example is indeed "toy" --- the echoed pulse and the original don't temporally overlap at all, so we could just zero out the echo to solve the problem. In the real setup, however, $k$ will be small compared to the signal length and the echo and original will be superimposed. In that case, if we zeroed out the samples, we'd be cutting out data, too, and our song might sound bad or just be missing a few seconds of music altogether. Still, this is a great example to test with, since it'll be obvious if we succeed in killing the echo.

Run the cell below, and see if you pass the sanity check. The plots for $x$ and $\hat{x}$ should be identical (minus trivial differences like color).

In [ ]:
x_hat = deconvolve(y, h_pad)

plt.figure(figsize=(16, 4))

plt.subplot(1, 3, 1); plt.stem(x_pad); plt.title("Original signal x")
plt.subplot(1, 3, 2); plt.stem(y, linefmt="C2", markerfmt="C2o"); plt.title("Echoed version y")
plt.subplot(1, 3, 3); plt.stem(x_hat, linefmt="C4", markerfmt="C4o"); plt.title("Recovered version x_hat")

plt.show()

## Q1c: Echo Removal

Implement `cancel_echo` below, which removes an echo of strength $\alpha$ and sample delay $k$ from the signal `x_echo`. We want `cancel_echo(transmit(x, alpha, k), alpha, k)` to return `x` (possibly with extra zeros on the end, which are harmless) for any valid choices of $\alpha, k$. 

Don't forget that:
1. We must again renormalize the final output audio matrix to 16-bit integer values the way we did in `transmit`.
2. The two audio channels must be treated as separate 1D signals. 

**Hint:** In `deconvolve`, the FFT vectors we divide must be the same length. This means that unlike in `transmit`, where you only defined the impulse response over $k+1$ indices, you should zero pad it to the same length as `x_echo` before doing the deconvolutions. An example of this is in the Q1b sanity check.

In [ ]:
def cancel_echo(x_echo, alpha, k):
    """
    Cancel an alpha-strength, k-sample delay echo from a two-channel audio signal x_echo
    where k > 0 and 0 < alpha < 1.
    
    Parameters:
    x_echo     - The echo-corrupted audio signal. Assumed to be an Nx2 matrix, where N is the number of audio samples.
    alpha      - The attenuation factor. Assumed that 0 < alpha < 1.
    k          - The delay factor, in samples. Assumed that k > 0.
    
    Returns:
    x_echoless - The signal with the echo cancelled.
    """
    # TODO


Now, run the cell below to see how well your echo cancellation algorithm works! If it's correct:
- The audio file will be 12 seconds long.
- The first 10 seconds will be the original copy of the song.
- The last 2 seconds will be empty, as those audio samples are all zeros. Since we cancelled the echo, which was the only thing present at the end of our echoed recording, there's now no music there. Don't worry about these data-less samples, they're harmless. We could crop them to get the exact same signal if we really wanted to, but it doesn't matter as we don't hear anything.

In [ ]:
x_cleaned = cancel_echo(x_echo, .8, 2*fs)
wavfile.write("wavs/echoless.wav", fs, x_cleaned)
ipd.Audio("wavs/echoless.wav")

Once you've successfully removed the echo, move on to Q1d.

## Q1d: Noisy Deconvolution

We removed an echo from a *clean* audio recording, which is no small feat! However, we haven't accounted for noise—we've assumed that the only unwanted effect that occurs is due to the echo itself. Fourier deconvolution's main flaw is that it tends to amplify noise, a problem which we'll now explore.

### Noise Model

We'll assume an *additive* noise model: after the parasitic convolution, there is a *noise signal*, which we'll denote as $z$, that is added to the final signal just before measurement. Below is a block diagram. Note that unlike $x$ and $h$, $z$ is random, although we won't dive too much into that aspect.

<img src="figs/deconv_noise_model.png" alt="drawing" style="width:500px;"/>

Now, we assume $y(n) = (x * h)(n) + z(n)$, and we again want to recover $x$ given only $y, h$. To get a sense for how a noised, echoed signal differs from a noiseless one, let's add a small amount of white Gaussian noise to `x_echo`, the signal from before.

In [ ]:
x_echo_noisy = x_echo + 800 * np.random.randn(x_echo.shape[0], x_echo.shape[1])
x_echo_noisy = np.int16(x_echo_noisy / np.max(np.abs(x_echo_noisy)) * 32767)

wavfile.write("wavs/echoed_noised_TakeOnMe.wav", fs, x_echo_noisy)
ipd.Audio("wavs/echoed_noised_TakeOnMe.wav")

**Q:** How does the noised, echoed version sound (the one we get by running the first cell in this part) compared to the echoed version from before? You should still hear the echo, but there's also white noise that's been added. What does the white noise sound like?

<span style="color:blue">**A:** (TODO)</span>

Now, we'll call the same echo cancellation function from before, and see what happens.

In [ ]:
x_cleaned_noisy = cancel_echo(x_echo_noisy, .8, 2*fs)
wavfile.write("wavs/echoless_noisy.wav", fs, x_cleaned_noisy)
ipd.Audio("wavs/echoless_noisy.wav")

**Q:** You should hear that the echo was removed, just as in the noiseless case. What about the noise, though? Is it louder or softer than before the deconvolution?

<span style="color:blue">**A:** (TODO)</span>

### Analyzing Noisy Deconvolution

Just as understanding how to deconvolve signals is most easily done in the frequency domain, so is performing an error analysis. We know that 

$$y(n) = (x * h)(n) + z(n) \implies Y(\omega) = X(\omega)H(\omega) + Z(\omega).$$

The Fourier deconvolution algorithm returns $\hat{X}(\omega)$, an estimate of $X(\omega)$, which is computed as 

$$\hat{X}(\omega) = \dfrac{Y(\omega)}{H(\omega)} = \dfrac{X(\omega)H(\omega) + Z(\omega)}{H(\omega)} = X(\omega) + \dfrac{Z(\omega)}{H(\omega)},$$

so the difference between our estimate, $\hat{X}$, and the true spectrum, $X$, is

$$\hat{X}(\omega) - X(\omega) = \dfrac{Z(\omega)}{H(\omega)}.$$

In general, this will be a complex number, which isn't very useful as a way to quantify error. Instead, we can consider the *magnitude* of the error between the estimated spectrum and the true one:

$$|\hat{X}(\omega) - X(\omega)| = \dfrac{|Z(\omega)|}{|H(\omega)|}$$.

As a final sanity check on your understanding of how Fourier deconvolution performs in the presence of noise, answer the following questions. 

Assume that for all $\omega$, $|Z(\omega)| = \sigma$, which roughly says that the noise is "equally strong" at all frequencies. Truly, $z$ is random, so it's wrong to think of $Z$ as a DTFT in the typical sense, but we'll defer those details to EECS 225A. Our analysis below still gets at the heart of the issue and the tradeoffs involved.

**Q:** Suppose $\sigma = 0$. Can we perfectly recover $X$, assuming that $|H(\omega)| > 0\ \forall \omega$? Explain.

<span style="color:blue">**A:** (TODO)</span>

**Q:** As $\sigma$ increases, intuitively, is our noise getting stronger or weaker? Is it easier, or more difficult, to recover $X$?

<span style="color:blue">**A:** (TODO)</span>

**Q:** The impulse response of our degradation system, which caused the echo, is $h(n) = \delta(n) + \alpha \delta(n -k)$, and so its frequency response is $H(\omega) = 1 + \alpha e^{-i\omega k}$. For concreteness, take $k=1$ and $\alpha=.8$. 

Which range of frequencies do you expect the noise amplification to be worst at: high ones, or low ones? Why? (*Hint*: Compute $|H(0)|$ and $|H(\pi)|$).

<span style="color:blue">**A:** (TODO) </span>

# References

[1] *Signal reflection (Wikipedia)*. [Link](https://en.wikipedia.org/wiki/Signal_reflection)  
[2] *AT&T Archives: Similarities of Wave Behavior*. [Link](https://www.youtube.com/watch?v=DovunOxlY1k)  
[3] *Audio feedback (Wikipedia)*. [Link](https://en.wikipedia.org/wiki/Audio_feedback)  
[4] *Dereverberation (Wikipedia)*. [Link](https://en.wikipedia.org/wiki/Dereverberation)  
[5] *Stereophonic Acoustic Echo Cancellation: Theory and Implementation*. [Link](http://lup.lub.lu.se/search/ws/files/4596819/1001945.pdf)  
[6] *Restoration of Hubble Space Telescope Images and Spectra*. [Link](http://www.stsci.edu/hst/HST_overview/documents/RestorationofHSTImagesandSpectra.pdf)  
[7] *Richardson-Lucy deconvolution (Wikipedia)*. [Link](https://en.wikipedia.org/wiki/Richardson%E2%80%93Lucy_deconvolution)  
[8] *Wiener deconvolution (Wikipedia)*. [Link](https://en.wikipedia.org/wiki/Wiener_deconvolution)  
[9] *Signals, Systems, and Inference, Chapter 11: Wiener Filtering*. [Link](https://ocw.mit.edu/courses/electrical-engineering-and-computer-science/6-011-introduction-to-communication-control-and-signal-processing-spring-2010/readings/MIT6_011S10_chap11.pdf)  
[10] 2D convolution GIF. [Link](https://upload.wikimedia.org/wikipedia/commons/4/4f/3D_Convolution_Animation.gif)  
[11] *LTI Models and Convolution, Section 11.2.3: Deconvolution*. [Link](https://ocw.mit.edu/courses/electrical-engineering-and-computer-science/6-02-introduction-to-eecs-ii-digital-communication-systems-fall-2012/readings/MIT6_02F12_chap11.pdf)  
[12] *The Scientist and Engineer's Guide to Digital Signal Processing: Chapter 17, Custom Filters and Deconvolution*. [Link](https://www.dspguide.com/ch17/2.htm)  
[13] *Statistics of natural image categories*. [Link](http://web.mit.edu/torralba/www/ne3302.pdf)